**Задание 1. Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, которая будет добавлять только новые вакансии/продукты в вашу базу.**

In [71]:
from bs4 import BeautifulSoup as bs
from pprint import pprint
from pymongo import MongoClient
import json
import re
import requests
from time import sleep

In [72]:
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'

URL_FIRST_PAGE_HH = 'https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=python&no_magic=true&L_save_area=true&items_on_page=20'

In [73]:
headers = {
    'User-Agent': USER_AGENT,
}

In [74]:
def parse_hh(url_page, headers, result=[], index_page=1):
    response = requests.get(url_page, headers=headers)
    if response.status_code != 200:
      print('Парсинг завершен')
      return result
    else:
      print('Cтраница №%d, ссылка: %s'%(index_page, response.url))

    dom = bs(response.content, 'html.parser')
    vacancies = dom.find_all('div', {'class': 'vacancy-serp-item__layout'})
    for vacancy in vacancies:
      result.append(parse_vacancy_hh(vacancy))

    link_next_page = dom.find('a', {'data-qa': 'pager-next'})
    if link_next_page:
      link_next_page =  'https://spb.hh.ru' + link_next_page['href']
    else:
      print('Парсинг завершен')
      return result

    result = parse_hh(link_next_page, headers, result, index_page+1)   
    return result
     
def parse_vacancy_hh(dom_vacancy):
  sleep(0.1)
  vacancy_name = dom_vacancy.find('a').text

  vacancy_salary = dom_vacancy.find('span', {'class', 'bloko-header-section-3'})
  if vacancy_salary:
    vacancy_salary = vacancy_salary.text
    min_salary, max_salary, currency_salary = clean_salary(vacancy_salary)
  else:
    min_salary, max_salary, currency_salary = None, None, None

  vacancy_link = dom_vacancy.find('a')['href']
  
  return {
      'vacancy_name': vacancy_name,
      'vacancy_salary': vacancy_salary,
      'min_salary': min_salary,
      'max_salary': max_salary,
      'currency_salary': currency_salary,
      'vacancy_link': vacancy_link,
      'vacancy_source': 'hh.ru',
  }

def clean_salary(vacancy_salary_text, min_salary=None, max_salary=None, currency_salary=None):
  list_salary = vacancy_salary_text.replace('\u202f', '').split()
  for i in range(len(list_salary) - 1):
    if list_salary[i] == 'от':
      min_salary = int(list_salary[i + 1])
    elif list_salary[i] == 'до':
      max_salary = int(list_salary[i + 1])
    elif list_salary[i] == '–':
      min_salary = int(list_salary[i - 1])
      max_salary = int(list_salary[i + 1])
  currency_salary = list_salary[-1]

  return min_salary, max_salary, currency_salary

In [75]:
result = parse_hh(URL_FIRST_PAGE_HH, headers)

Cтраница №1, ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=python&no_magic=true&L_save_area=true&items_on_page=20
Cтраница №2, ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=python&no_magic=true&L_save_area=true&items_on_page=20&page=1&hhtmFrom=vacancy_search_list
Cтраница №3, ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=python&no_magic=true&L_save_area=true&items_on_page=20&page=2&hhtmFrom=vacancy_search_list
Cтраница №4, ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=python&no_magic=true&L_save_area=true&items_on_page=20&page=3&hhtmFrom=vacancy_search_list
Cтраница №5, ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=descripti

In [76]:
len(result)

167

In [77]:
client = MongoClient()
db = client.vacancies_python_hh

In [78]:
collection_vacancies_hh_ru = db.hh_ru

In [79]:
def cheak_and_save_vacancies_in_db(vacancies):
  for vacancy in vacancies:
    if not len(list(collection_vacancies_hh_ru.find({'vacancy_link': vacancy['vacancy_link']}))):
      collection_vacancies_hh_ru.insert_one(vacancy)

In [80]:
cheak_and_save_vacancies_in_db(result)

In [81]:
result_find = list(collection_vacancies_hh_ru.find())

In [82]:
len(result_find)

168

In [83]:
result_find

[{'_id': ObjectId('63f891c440b1df7cd2c5ed6a'),
  'vacancy_name': 'Преподаватель Python',
  'vacancy_salary': 'от 125\u202f000 руб.',
  'min_salary': 125000,
  'max_salary': None,
  'currency_salary': 'руб.',
  'vacancy_link': 'https://spb.hh.ru/vacancy/77083921?from=vacancy_search_list&query=python',
  'vacancy_source': 'hh.ru'},
 {'_id': ObjectId('63f891c440b1df7cd2c5ed6b'),
  'vacancy_name': 'Разработчик Python',
  'vacancy_salary': 'от 120\u202f000 руб.',
  'min_salary': 120000,
  'max_salary': None,
  'currency_salary': 'руб.',
  'vacancy_link': 'https://spb.hh.ru/vacancy/76076341?from=vacancy_search_list&query=python',
  'vacancy_source': 'hh.ru'},
 {'_id': ObjectId('63f891c440b1df7cd2c5ed6c'),
  'vacancy_name': 'Lead Python developer',
  'vacancy_salary': 'до 500\u202f000 руб.',
  'min_salary': None,
  'max_salary': 500000,
  'currency_salary': 'руб.',
  'vacancy_link': 'https://spb.hh.ru/vacancy/77337810?from=vacancy_search_list&query=python',
  'vacancy_source': 'hh.ru'},
 {'_i

In [84]:
result_find = collection_vacancies_hh_ru.find()
for item in result_find:
    print(item)

{'_id': ObjectId('63f891c440b1df7cd2c5ed6a'), 'vacancy_name': 'Преподаватель Python', 'vacancy_salary': 'от 125\u202f000 руб.', 'min_salary': 125000, 'max_salary': None, 'currency_salary': 'руб.', 'vacancy_link': 'https://spb.hh.ru/vacancy/77083921?from=vacancy_search_list&query=python', 'vacancy_source': 'hh.ru'}
{'_id': ObjectId('63f891c440b1df7cd2c5ed6b'), 'vacancy_name': 'Разработчик Python', 'vacancy_salary': 'от 120\u202f000 руб.', 'min_salary': 120000, 'max_salary': None, 'currency_salary': 'руб.', 'vacancy_link': 'https://spb.hh.ru/vacancy/76076341?from=vacancy_search_list&query=python', 'vacancy_source': 'hh.ru'}
{'_id': ObjectId('63f891c440b1df7cd2c5ed6c'), 'vacancy_name': 'Lead Python developer', 'vacancy_salary': 'до 500\u202f000 руб.', 'min_salary': None, 'max_salary': 500000, 'currency_salary': 'руб.', 'vacancy_link': 'https://spb.hh.ru/vacancy/77337810?from=vacancy_search_list&query=python', 'vacancy_source': 'hh.ru'}
{'_id': ObjectId('63f891c440b1df7cd2c5ed6d'), 'vacanc

**Задание 2.Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты). То есть цифра вводится одна, а запрос проверяет оба поля**

In [105]:
praise = 100000

In [106]:
for item in collection_vacancies_hh_ru.find({'$or' :  [{'min_salary': {'$gt': praise}},
                                                         {'max_salary': {'$gt': praise}}]}):
    print(item)

{'_id': ObjectId('63f891c440b1df7cd2c5ed6a'), 'vacancy_name': 'Преподаватель Python', 'vacancy_salary': 'от 125\u202f000 руб.', 'min_salary': 125000, 'max_salary': None, 'currency_salary': 'руб.', 'vacancy_link': 'https://spb.hh.ru/vacancy/77083921?from=vacancy_search_list&query=python', 'vacancy_source': 'hh.ru'}
{'_id': ObjectId('63f891c440b1df7cd2c5ed6b'), 'vacancy_name': 'Разработчик Python', 'vacancy_salary': 'от 120\u202f000 руб.', 'min_salary': 120000, 'max_salary': None, 'currency_salary': 'руб.', 'vacancy_link': 'https://spb.hh.ru/vacancy/76076341?from=vacancy_search_list&query=python', 'vacancy_source': 'hh.ru'}
{'_id': ObjectId('63f891c440b1df7cd2c5ed6c'), 'vacancy_name': 'Lead Python developer', 'vacancy_salary': 'до 500\u202f000 руб.', 'min_salary': None, 'max_salary': 500000, 'currency_salary': 'руб.', 'vacancy_link': 'https://spb.hh.ru/vacancy/77337810?from=vacancy_search_list&query=python', 'vacancy_source': 'hh.ru'}
{'_id': ObjectId('63f891c440b1df7cd2c5ed6d'), 'vacanc